In [16]:
from openai import OpenAI
import os
import json
import time
import ast
client = OpenAI(api_key=os.environ["OPENAI_API_KEY"])
assistant = client.beta.assistants.retrieve(assistant_id='asst_HBm9X29io6tMuiphtvQeTuyh')
thread = client.beta.threads.create()
ASSISTANT_ID = assistant.id

In [9]:
submit_message(client, ASSISTANT_ID, thread, "Hello, I'm a chatbot created by OpenAI. How can I help you today?")

Run(id='run_H6Jng6yk3zF7OZn6lwV4aUkq', assistant_id='asst_HBm9X29io6tMuiphtvQeTuyh', cancelled_at=None, completed_at=None, created_at=1711016190, expires_at=1711016790, failed_at=None, file_ids=['file-PgCvGp2KSV0xsnZe7O4P2PGv'], instructions='You are an assistant that is supposed to answer user questions regarding Kanton Saint Gallen, Switzerland. You have access to all the data in the given text file from their website.\n\nIf you don\'t know the answer, you can call the function to "forward call" to one of the departments. You can get the department instructions from the department.txt file.\n\nAnswer is "Swiss" German if the user talks in German.', last_error=None, metadata={}, model='gpt-4-turbo-preview', object='thread.run', required_action=None, started_at=None, status='queued', thread_id='thread_7YcEruvuCHxeEFxZSLGltd0L', tools=[ToolAssistantToolsRetrieval(type='retrieval'), ToolAssistantToolsFunction(function=FunctionDefinition(name='forward_call', description='Forward the call 

In [25]:
def submit_message(client,assistant_id, thread, user_message):
    client.beta.threads.messages.create(
        thread_id=thread.id, role="user", content=user_message
    )
    return client.beta.threads.runs.create(
        thread_id=thread.id,
        assistant_id=assistant_id,
    )

def get_response(client,thread):
    return client.beta.threads.messages.list(thread_id=thread.id, order="asc")

def pretty_print(messages):
    result = ""
    for m in messages.data[-1:]:
        result += f"{m.content[0].text.value}\n"
    result += "\n"
    return result

# Waiting in a loop
def wait_on_run(client,run, thread):
    while run.status == "queued" or run.status == "in_progress":
        run = client.beta.threads.runs.retrieve(
            thread_id=thread.id,
            run_id=run.id,
        )
        time.sleep(0.5)
    return run


def check_response(client,thread,run):
        # Extract single tool call
    tool_call = run.required_action.submit_tool_outputs.tool_calls[0]
    name = tool_call.function.name
    arguments = json.loads(tool_call.function.arguments)

    response = {'success': "Forwarding the call to the department. Ask user to hold"}

    run = client.beta.threads.runs.submit_tool_outputs(
        thread_id=thread.id,
        run_id=run.id,
        tool_outputs=[
            {
                "tool_call_id": tool_call.id,
                "output": json.dumps(response),
            }
        ],
    )
    
    run = wait_on_run(client,run, thread)
    return get_response(client,thread)

In [32]:
run = submit_message(client,ASSISTANT_ID, thread, "Can you connect me to the city department?")
run = wait_on_run(client,run, thread)
if run.status == 'requires_action':
    response = pretty_print(check_response(client,thread,run))
    print(response)
else:
    response = pretty_print(get_response(client,thread))
    print(response)

I've forwarded your request to the City Department. Please hold while they get back to you.


